In [1]:
from random import seed
from random import random
import numpy as np
 
# 네트워크 초기 설정
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [2]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i] # 각 인풋과 가중치의 곱의 합
    return activation

def sigmoid(activation):
    return 1/(1+np.exp(-activation)) # 시그모이드 값 리턴

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs) # 결과를 만들고
            neuron['output'] = sigmoid(activation) # 시그모이드 처리 후
            new_inputs.append(neuron['output']) # 새로운 인풋을 정의하자
        inputs = new_inputs
    return inputs

**여기까지는 순전파 학습과정이었습니다. 이 과정이 끝나면 가중치가 바뀌나요?  
답변을 답변의 근거 코딩 결과와 함께 보여주세요.**

- 가중치는 바뀌지 않았습니다.

In [3]:
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [4]:
def sigmoid_derivative(output):
    result = sigmoid(output) * (1 - sigmoid(output))
    return result

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error) 
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output']) # 함수 처음은 else 문의 이 for문에 들어옴. 이때는 예측값과 뉴런의 아웃풋의 차로 오차를 지정
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * sigmoid_derivative(neuron['output']) # 오차와 아웃풋의 미분값의 곱으로 델타를 정의


In [5]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002388909426147501}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14879067689118655}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.06037599461732701}]


In [6]:
def weights_update(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j] # 각 웨이트 업데이트! I_rate와 델타, 인풋의 곱으로
            neuron['weights'][-1] += l_rate * neuron['delta'] # 절편은 인풋을 곱해줄 필요가 없어용
            

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            
            for i in range(len(expected)):
                sum_error += (expected[i] - outputs[i]) ** 2 # 차의 제곱들의 합
            
            backward_propagate_error(network, expected)
            weights_update(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [7]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [9]:
n_inputs = len(dataset[0])-1 # 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는? # 맨 마지막 로우인덱스 제외한 길이!
temp = []
for row in dataset:
    temp.append(row[-1])
n_outputs = len(set(temp)) # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠?  # 0 또는 1
network = initialize_network(n_inputs, 2, n_outputs)

train_network(network, dataset, 0.5, 10, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.

# 학습된(최적화)된 네트워크가 초기 네트워크와 달리 어떻게 변하였는지 출력하시오.
for layer in network:
    print(layer)

>epoch=0, lrate=0.500, error=6.089
>epoch=1, lrate=0.500, error=5.067
>epoch=2, lrate=0.500, error=4.459
>epoch=3, lrate=0.500, error=4.133
>epoch=4, lrate=0.500, error=4.238
>epoch=5, lrate=0.500, error=4.453
>epoch=6, lrate=0.500, error=4.614
>epoch=7, lrate=0.500, error=4.741
>epoch=8, lrate=0.500, error=4.846
>epoch=9, lrate=0.500, error=4.931
[{'weights': [-4.97739131434333, 1.4209381045340803, 0.6505630061513098], 'output': 4.387224866432003e-14, 'delta': -0.049913199746922376}, {'weights': [-2.2116843512273814, 0.48123017741165636, 0.3032425987754865], 'output': 9.145489851671995e-07, 'delta': -0.02979156651368742}]
[{'weights': [0.922427535367579, 1.0789579071428408, -0.32243991631181057], 'output': 0.43270944456393584, 'delta': -0.10326757533964233}, {'weights': [-0.9614895772748131, -0.07133064366874067, 0.19045075534453118], 'output': 0.5339880142567771, 'delta': 0.1085773017734112}]


In [14]:
# 학습한 네트워크로 예측값을 뽑아보자.

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [15]:
# 네트워크가 잘 학습되었는지 확인해보자. 

for row in dataset:
    prediction = predict(network, row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    print('실제값=%d, 예측값=%d' % (row[-1], prediction))

실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
